### ¿Qué son los métodos Wrapper?

Los métodos **Wrapper** evalúan subconjuntos de características utilizando un algoritmo de aprendizaje específico como una "caja negra". El subconjunto se selecciona basándose en el rendimiento del modelo (ej. precisión).

**Implementación en Python (SFS):**
La herramienta principal es `SequentialFeatureSelector` de `sklearn.feature_selection`.

#### 1. Métodos de Modelación Compatibles
Puedes usar cualquier estimador de scikit-learn que tenga los métodos `.fit()` y `.predict()` (o `.transform()`):
*   **Clasificación:** `LogisticRegression`, `RandomForestClassifier`, `SVC`, `KNeighborsClassifier`, `GradientBoostingClassifier`.
*   **Regresión:** `LinearRegression`, `SVR`, `RandomForestRegressor`, `Lasso`.

#### 2. Métricas de Scoring (Parámetro `scoring`)
Dependen del tipo de problema:
*   **Clasificación:** `'accuracy'`, `'f1'`, `'precision'`, `'recall'`, `'roc_auc'`.
*   **Regresión:** `'r2'`, `'neg_mean_squared_error'`, `'neg_mean_absolute_error'`.

#### 3. Métodos de Búsqueda
*   **Forward (Hacia adelante):** Comienza con 0 variables y añade la que más mejora el modelo en cada paso.
*   **Backward (Hacia atrás):** Comienza con todas las variables y elimina la menos importante en cada paso.

**Ejemplo de flujo:**
```python
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier

# Definir modelo
modelo = RandomForestClassifier()

# Configurar selector
sfs = SequentialFeatureSelector(modelo, n_features_to_select=3, direction='forward', scoring='accuracy', cv=5)

# Ejecutar
sfs.fit(X, y)
```

In [2]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import KFold
from sklearn.base import clone
import pandas as pd

def wrapper_feature_selection(
    X,
    y,
    model,
    search_method='forward',
    k_features=20,
    cv=10,
    scoring='r2',
    n_jobs=-1
):
    kf = KFold(n_splits=cv, shuffle=True, random_state=42)
    estimator = clone(model)

    if search_method in ['forward', 'backward']:
        selector = SequentialFeatureSelector(
            estimator=estimator,
            n_features_to_select=k_features,
            direction=search_method,
            scoring=scoring,
            cv=kf,
            n_jobs=n_jobs
        )
    else:
        raise ValueError("search_method must be: forward or backward")

    selector.fit(X, y)
    selected_features = X.columns[selector.get_support()]

    return {
        "selected_features": selected_features,
        "X_selected": X[selected_features],
        "selector": selector
    }

In [5]:
from sklearn.datasets import load_wine
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

# 1. Cargar el dataset de Wine
wine = load_wine()
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y = wine.target

# 2. Definir los experimentos extendidos
modelos = {
    'LogisticRegression': LogisticRegression(max_iter=100),
    'RandomForest': RandomForestClassifier(n_estimators=50, random_state=42),
    'SVM': SVC(kernel='linear'),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=50)
}

metodos = ['forward', 'backward']
metricas = ['accuracy', 'f1']

resultados_lista = []

print("Iniciando experimentos extendidos...")

# 3. Iterar y ejecutar el wrapper
for mod_name, model in modelos.items():
    for met in metodos:
        for score in metricas:
            print(f"Ejecutando: {mod_name} | {met} | {score}")
            try:
                res = wrapper_feature_selection(
                    X=X, y=y, model=model,
                    search_method=met, k_features=5,
                    cv=10, scoring=score
                )
                resultados_lista.append({
                    'Modelo': mod_name,
                    'Metodo': met,
                    'Scoring': score,
                    'Features': ", ".join(res['selected_features'].tolist())
                })
            except Exception as e:
                print(f"Error en {mod_name} con {score}: {e}")

# 4. Mostrar resultados
df_resultados = pd.DataFrame(resultados_lista)
display(df_resultados)

Iniciando experimentos extendidos...
Ejecutando: LogisticRegression | forward | accuracy
Ejecutando: LogisticRegression | forward | f1
Ejecutando: LogisticRegression | backward | accuracy
Ejecutando: LogisticRegression | backward | f1
Ejecutando: RandomForest | forward | accuracy
Ejecutando: RandomForest | forward | f1
Ejecutando: RandomForest | backward | accuracy
Ejecutando: RandomForest | backward | f1
Ejecutando: SVM | forward | accuracy
Ejecutando: SVM | forward | f1
Ejecutando: SVM | backward | accuracy
Ejecutando: SVM | backward | f1
Ejecutando: GradientBoosting | forward | accuracy
Ejecutando: GradientBoosting | forward | f1
Ejecutando: GradientBoosting | backward | accuracy
Ejecutando: GradientBoosting | backward | f1


,Modelo,Metodo,Scoring,Features
0,LogisticRegression,forward,accuracy,"alcohol, ash, alcalinity_of_ash, flavanoids, hue"
1,LogisticRegression,forward,f1,"alcohol, malic_acid, ash, alcalinity_of_ash, m..."
2,LogisticRegression,backward,accuracy,"alcalinity_of_ash, flavanoids, color_intensity..."
3,LogisticRegression,backward,f1,"proanthocyanins, color_intensity, hue, od280/o..."
4,RandomForest,forward,accuracy,"alcohol, magnesium, flavanoids, color_intensit..."
5,RandomForest,forward,f1,"alcohol, malic_acid, ash, alcalinity_of_ash, m..."
6,RandomForest,backward,accuracy,"alcohol, magnesium, flavanoids, color_intensit..."
7,RandomForest,backward,f1,"proanthocyanins, color_intensity, hue, od280/o..."
8,SVM,forward,accuracy,"alcohol, ash, alcalinity_of_ash, flavanoids, o..."
9,SVM,forward,f1,"alcohol, malic_acid, ash, alcalinity_of_ash, m..."
